In [ ]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
import pytz, os,json, requests
import pandas as pd
from datetime import datetime, timedelta
from io import BytesIO
from googleapiclient.errors import HttpError
import googleapiclient.http

# Path to your service account key file and the scopes needed for the Drive API
SERVICE_ACCOUNT_FILE = "/Users/sachin/TheJuniorDataScientist/credentials/sachin_service account.json"
SCOPES = ['https://www.googleapis.com/auth/drive']

# Authenticate and build the Drive API client
def authenticate_drive_service():
    try:
        credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
        drive_service = build('drive', 'v3', credentials=credentials)
        print("Google Drive service authorized successfully.")
        return drive_service
    except Exception as e:
        print("Error during authentication:", e)
        raise

# Generate date range (YYYYMMDD format)
def get_date_range(local_tz, start_date=None, end_date=None, days=None):
    current_time = datetime.now(pytz.utc).astimezone(local_tz)
    if end_date:
        end_date = datetime.strptime(end_date, "%Y%m%d").astimezone(local_tz)
    else:
        end_date = current_time

    if start_date:
        start_date = datetime.strptime(start_date, "%Y%m%d").astimezone(local_tz)
    elif days:
        start_date = end_date - timedelta(days=days)
    else:
        start_date = end_date - timedelta(days=10)

    return start_date.strftime("%Y%m%d"), end_date.strftime("%Y%m%d")

def fetch_files_with_dates(drive_service, folder_id, date_range):
    try:
        query = f"'{folder_id}' in parents and mimeType='text/csv' and trashed=false"
        
        files = []
        page_token = None  # Initialize page token
        
        while True:
            # Fetch a page of files
            response = drive_service.files().list(
                q=query,
                fields="nextPageToken, files(id, name, size)",
                pageToken=page_token
            ).execute()

            # Extend the list with new files from this page
            files.extend(response.get('files', []))

            # Check if there's another page
            page_token = response.get('nextPageToken')
            if not page_token:
                break  # No more pages, exit loop
        
        print(f"Total files found in the folder: {len(files)}")

        # Filter files by date in filenames
        filtered_files = []
        for file in files:
            for date in date_range:
                if date in file['name']:
                    file['size_mb'] = round(int(file.get('size', 0)) / (1024 * 1024), 2)  # Convert size to MB
                    filtered_files.append(file)
                    break

        print(f"Number of files matching the date range: {len(filtered_files)}")
        return filtered_files
    except HttpError as error:
        print(f"An error occurred: {error}")
        raise



def process_files(drive_service, files, required_columns, checkpoint_file="download_progress.json", save_interval = 50 * 1024 * 1024, chunksize=10 * 1024 * 1024):
    combined_data = []
    last_save_time = datetime.now()

    try:
        with open(checkpoint_file, "r") as f:
            download_progress = json.load(f)
    except (FileNotFoundError, json.JSONDecodeError):
        download_progress = {}

    for i, file in enumerate(files):
        try:
            file_id = file['id']
            file_name = file['name']
            file_size = int(file.get('size', 0))
            bytes_downloaded = download_progress.get(file_name, 0)

            if bytes_downloaded >= file_size:
                print(f"{file_name} has already been downloaded. Skipping...")
                continue
            
            print(f"\nFile number {i+1} - Downloading file: {file_name} (Resuming from {bytes_downloaded} bytes)")

            request = drive_service.files().get_media(fileId=file_id)
            file_stream = BytesIO()
            downloader = googleapiclient.http.MediaIoBaseDownload(file_stream, request, chunksize=chunksize)

            done = False
            while not done:
                try:
                    status, done = downloader.next_chunk()
                    bytes_downloaded += len(file_stream.getvalue())
                    download_progress[file_name] = bytes_downloaded

                    if (datetime.now() - last_save_time).total_seconds() > 60 or bytes_downloaded % save_interval < chunksize:
                        with open(checkpoint_file, "w") as f:
                            json.dump(download_progress, f)
                        last_save_time = datetime.now()

                    print(f"Download {int(status.progress() * 100)}% complete for {file_name}.")
                except (requests.ConnectionError, HttpError) as e:
                    print(f"Network error during download: {e}. Retrying...")
                    break

            file_stream.seek(0)
            df = pd.read_csv(file_stream, usecols=required_columns)
            df = df[required_columns]
            df['source_file'] = file_name
            combined_data.append(df)

            if file_name in download_progress:
                del download_progress[file_name]
                with open(checkpoint_file, "w") as f:
                    json.dump(download_progress, f)

        except Exception as e:
            print(f"Error processing file {file_name}: {e}")

    if combined_data:
        combined_data = [df for df in combined_data if not df.empty]
        if combined_data:
            final_df = pd.concat(combined_data, ignore_index=True)
            final_df[['to_qty']] = final_df[['to_qty']].fillna(0)
            print("All files processed and merged successfully.")
            return final_df
    else:
        print("No files were successfully processed.")
        return pd.DataFrame()


# Step 1: Authenticate Drive Service
drive_service = authenticate_drive_service()

local_tz = pytz.timezone("Asia/Kolkata")
start_date_input = None # "20250201"
end_date_input =  None #"20250225"
days_range = 5

start_date, end_date = get_date_range(local_tz, start_date_input, end_date_input, days_range)
date_range = pd.date_range(start=start_date, end=end_date).strftime("%Y%m%d").tolist()
print(f"Date range for filtering: {start_date} to {end_date}")

# Step 3: Fetch Files Matching the Date Range
folder_id = '1akFe2_iKCqoZ0lAETRipXxDSKYnFmx2x'
filtered_files = fetch_files_with_dates(drive_service, folder_id, date_range)
file_df = pd.DataFrame(filtered_files)
file_df.drop(columns=['size','id'],inplace=True, axis=1)
print(file_df, f"\n\nTotal Data to be processed {file_df['size_mb'].sum()} MB")

# Step 4: Process Files and Combine Data
required_columns = ['dd', 'mm', 'yyyy', 'mh_code', 'type', 'store_name', 'to_qty','store_id', 'product_variant_id']
final_df = process_files(drive_service, filtered_files, required_columns)

print('Total rows before removing null values :',final_df.shape[0])
### Printing Null Records
#final_df[final_df[['dd', 'mm', 'yyyy', 'mh_code', 'type', 'store_name', 'store_id']].isna().all(axis=1)]




# Dropping Null records
main_df = final_df.dropna(subset=['dd', 'mm', 'yyyy', 'mh_code', 'type', 'store_name', 'store_id', 'product_variant_id'], how='all').copy()
columns_to_convert = ['dd', 'mm', 'yyyy']
try:
    main_df[columns_to_convert] = main_df[columns_to_convert].fillna(1).astype(int)
    main_df.rename(columns={'yyyy': 'year', 'mm': 'month', 'dd': 'day'}, inplace=True)
except Exception as e:
    print(f"Error during  conversion: {e}")
try:
    # Creating the date column from dd, mm, yyyy
    main_df['date'] = pd.to_datetime(main_df[['year', 'month', 'day']])
    print("Date column created successfully!")
except Exception as e:
    print(f"Error during date conversion: {e}")


main_df = main_df.groupby(['type','mh_code', 'store_name', 'store_id', 'source_file', 'date', 'product_variant_id']).agg({'to_qty': 'sum'}).reset_index()
main_df.rename(columns={'store_name': 'Destination_store_name', 'store_id': 'Destination_store_id'}, inplace=True)


Google Drive service authorized successfully.
Date range for filtering: 20250309 to 20250314
Total files found in the folder: 4142
Number of files matching the date range: 34
                      name  size_mb
0         20250313_CHN.csv    13.15
1         20250313_KOL.csv     4.45
2         20250313_NCR.csv    38.53
3      20250313_Mumbai.csv    21.59
4         20250313_HYD.csv    16.48
5        20250313_Pune.csv     6.16
6   20250313_Bengaluru.csv    27.81
7         20250312_CHN.csv    13.09
8         20250312_NCR.csv    39.70
9         20250312_HYD.csv    15.42
10        20250312_KOL.csv     5.91
11       20250312_Pune.csv     6.24
12  20250312_Bengaluru.csv    28.10
13        20250311_HYD.csv    14.94
14        20250311_CHN.csv    11.48
15        20250311_KOL.csv     4.76
16        20250311_NCR.csv    41.31
17  20250311_Bengaluru.csv    26.16
18     20250311_Mumbai.csv    18.48
19       20250311_Pune.csv     5.33
20        20250310_CHN.csv    11.66
21        20250310_KOL.csv     4.